In [ ]:
import os
import re
import sys
import time
import datetime
import psycopg2
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from matplotlib import pyplot as plt

In [ ]:
%matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'cypy2'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)

### Analyzing timepoints

In [ ]:
d = pd.read_sql(
    'select strava_date as date, heart_rate, power, cadence, speed from proc_records r'
    ' left join metadata m on m.activity_id = r.activity_id'
    ' where activity_type = \'ride\' and bike_name = \'giant-defy-advanced\'',
    conn)
d = d.dropna(axis=0, how='any')

In [ ]:
d['year'] = [t.year for t in d.date]
year = np.concatenate(tuple([[row.year] * len(row.speed) for ind, row in d.iterrows()]))
d.drop(['year'], axis=1, inplace=True)

In [ ]:
data = []
for column in d.columns:
    if column=='date': continue
    vals = np.concatenate(d[column].values)
    data.append(vals)
data = np.array(data)

In [ ]:
data.shape

In [ ]:
mask = np.any(pd.isna(data), axis=0)
data = data[:, ~mask].astype(float)
year = year[~mask]

In [ ]:
# hack to correct for fitparse bug
speed = data[3, :]
speed[speed > 30] /= 1000
data[3, :] = speed

In [ ]:
df = pd.DataFrame(data=data.transpose(), columns=['hrt', 'pwr', 'cad', 'spd'])
df['year'] = year

In [ ]:
hrt = df.hrt
power = df.pwr
hrt = hrt[power > 0]
power = power[power > 0]

_ = plt.hist2d(hrt, power, bins=[np.arange(80, 190, 2), np.arange(50, 400, 10)])

In [ ]:
bins= {
    'cad': np.arange(60, 90, 1),
    'pwr': np.arange(50, 400, 10),
    'spd': np.arange(2, 6, .03),
    'hrt': np.arange(80, 190, 2),
}

In [ ]:
def hist2d(df, x_col, y_col, year=None):
        
    df = df.copy()
    if year:
        df = df.loc[df.year==year]

    x_vals, y_vals = df[x_col], df[y_col]
    x_bins, y_bins = bins[x_col], bins[y_col]
    
    mask = (x_vals > min(x_bins)) & (x_vals < max(x_bins)) & (y_vals > min(y_bins)) & (y_vals < max(y_bins))
    x_vals, y_vals = x_vals[mask], y_vals[mask]

    h = plt.hist2d(x_vals, y_vals, bins=[x_bins, y_bins])
    hist = h[0]
    hist /= hist.sum()
    return hist.transpose()

In [ ]:
x_col, y_col = 'hrt', 'pwr'
hists = {year: hist2d(df, x_col, y_col, 2000 + year) for year in [16, 17, 18]}

In [ ]:
def autogain(im, minn=None, maxx=None):
    im = im.astype(float)
    
    if not minn:
        minn = im.min()
    if not maxx:
        maxx = im.max()
        
    im -= minn
    im[im < 0] = 0

    im /= (maxx - minn)
    im[im > 1] = 1

    return im

In [ ]:
blue, orange, green, red = sns.color_palette()[:4]

In [ ]:
def add_color(im_rgb, im_bw, color):
    h = autogain(im_bw)
    for ind, weight in enumerate(color):
        im_rgb[:, :, ind] += im_bw * weight
    return im_rgb

# im = np.concatenate((normz(h[16])[:, :, None], normz(h[17])[:, :, None], normz(h[18])[:, :, None]), axis=2)

In [ ]:
sz = hists[16].shape

extent = [min(bins[x_col]), max(bins[x_col]), min(bins[y_col]), max(bins[y_col])]

# initialize RGB image
im = np.zeros(list(sz) + [3])

for h, color in zip(hists.values(), [blue, green, red]):
    im = add_color(im, h, color)
    
plt.figure()
plt.imshow(autogain(im[::-1, :, :])**1, aspect='auto', interpolation=None, extent=extent)

In [ ]:
im = hists[18] - hists[17]
plt.imshow(autogain(im[::-1, :], minn=-1, maxx=1), aspect='auto', extent=extent)

In [ ]:
plt.imshow(np.log(h2016[0] + 1).transpose(), aspect='auto')

In [ ]:
year = 2016
cad = df.loc[df.year==year].cad
speed = df.loc[df.year==year].spd
speed = speed[cad > 0]
cad = cad[cad > 0]

fig, axs = plt.subplots(15, 1)
for ind, ax in enumerate(axs):
    h, edges = np.histogram(speed[cad==(60 + ind*2)], bins=np.arange(2, 6, .05))
    h = h/h.sum()
    ax.plot(edges[1:], h, linewidth=1)
    ax.set_ylim([0, .1])

### Strava-like heatmap

In [ ]:
d = dbutils.execute_query(conn,
    '''select ST_AsGeoJSON(ST_Collect(ST_Simplify(geom, .00001)))
        from proc_records, metadata
        where proc_records.activity_id = metadata.activity_id
        and metadata.activity_type = 'ride'
        and ST_Intersects(ST_MakeEnvelope(-122.3, 37.86, -122.15, 37.923, 4326), geom)
    ''')

In [ ]:
coords = json.loads(d[0][0])['coordinates']
coords_cat = np.concatenate(tuple([np.array(coord) for coord in coords]), axis=0)
coords_cat.shape

In [ ]:
for coord in coords:
    coord = np.array(coord)
    plt.plot(coord[:, 0], coord[:, 1])

In [ ]:
r = plt.hist2d(coords_cat[:, 0], coords_cat[:, 1], bins=900)

In [ ]:
plt.figure()
plt.imshow(np.log((r[0]).transpose()[::-1, :] + 1))